In [1]:
# example for reading DARDAR .h5 data files,
#  and putting multiple orbits onto a global quarter degree grid
import h5py
from mpl_toolkits.basemap import Basemap, cm
import numpy as np
import matplotlib.pyplot as plt
import os
#from os import listdir

In [ ]:
#dirt = '/home/dudavid/Dendrite/DAR/'
dirt = '/home/dudavid/Dendrite/Dendrite/SatData/DARDAR/2008/11/' # one day as example
#dirt = '/home/dudavid/Dendrite/Dendrite/SatData/DARDAR/2008/11/25/' # one day as example
#files = os.listdir(dirt)
days = os.listdir(dirt)
iwpall,lata,dna,dmask, iwpallb,iwpalls = [],[],[],[], [],[]
#iwpallb,latab,dna,dmask = [],[],[],[]
startlat = 70.0 # abs value of NS boundaries, should match with ny below
gsize = 1.0 # grid size in degree
nx, ny = int(360/gsize), int(startlat*2/gsize)
iwpg, iwpgc = np.zeros([ny,nx]), np.zeros([ny,nx]) # total,count
iwpgb, iwpgcb= np.zeros([ny,nx]),np.zeros([ny,nx]) # total,count

# try doing 6 pixel long average of DARDAR data
nsmo = 3 # double this add 1, so 7

In [ ]:
for d in range(len(days)):
#for d in range(3):
    files = os.listdir(dirt+days[d])
    print(files[0],d)
    #for fix in range(2):
    for fix in range(len(files)):
        #print(files[fix])
        file = dirt+days[d]+'/'+files[fix]

        f = h5py.File(file,'r')
        iw = np.array(f['iwc']) # IWC in kg/m^3
        flag = np.array(f['land_water_mask'])
        # mask is from calipso flags where:
        # (http://www.icare.univ-lille1.fr/projects_data/dardar/docs/varcloud-algorithm_description-v1.0.pdf)
        #  0=shallow ocean 1=land 2=coastlines 3=shallow inland water 4=intermittent water 
        #  5=deep inland water 6=coastal ocean 7=deep ocean
        hgt = np.array(f['height']) #height above sea level in m [436]
        hmask,dz = [], []
        for z in range(len(hgt)):
            if hgt[z]>=0 and hgt[z]<20000:  hmask.append(z)
        #        dz.append(hgt[z]-hgt[z+1]) #these are all 60m!
        mask = flag > 4
        iwc = iw[mask,:] # just profiles over ocean (for now)
        
        dmsk = np.array(f['DARMASK_Simplified_Categorization'])[mask] #as follows (same source doc):
        # -9 ground, -1 dunno, 0 clear, 1 ice, 2 ice+supercooled, 3 liquid (warm),
        #  4 supercooled, 5 rain, 6 aerosol, 7 maybe insects, 8 stratospheric feature
        lat = np.array(f['latitude'])[mask]
        lon = np.array(f['longitude'])[mask]
        dayn = np.array(f['day_night_flag'])[mask] #day=0, night=1
        #print('# profiles summing: ',len(iwc[:,20]))
        
        # calculate lat/lon indices for Xdeg grid:
        ladex = np.floor((lat[:]+startlat) / gsize)
        lodex = np.floor((lon[:]+180.0) / gsize)

        iwp = [sum(iwc[x,hmask]*60.0) for x in range(len(iwc[:,20]))]
        ariwp = np.array(iwp)
        # all range gates are 60m here, so kg/m3 * m yields IWP in kg/m2
        #print(info(np.array(iwp)))
        
        iwpall.extend(iwp)
        lata.extend(lat)
        dna.extend(dayn)
        dmask.extend(dmsk)
        
        # running mean over iwp series:
        iwpsmoo = ariwp # first few will be same, so will last few. 
        iwpsmoo[nsmo:(len(ariwp)-nsmo)] = \
         [ariwp[(x-nsmo):(x+nsmo)].mean() for x in range(nsmo,(len(ariwp)-nsmo))]
        #print(np.size(iwpsmoo),np.size(lodex))
        # broken up into chunks of 2*nsmo, then averaged
        l = 2*nsmo
        iwpb = np.array([ariwp[x*l:(x*l+2*nsmo)].mean() for x in range( int(np.floor(len(ariwp)/l)) )])
        ladb = np.array([lat[x*l:(x*l+2*nsmo)].mean() for x in range( int(np.floor(len(ariwp)/l)) )])
        lodb = np.array([lon[x*l:(x*l+2*nsmo)].mean() for x in range( int(np.floor(len(ariwp)/l)) )])
        ladexb = np.floor((ladb[:]+startlat) / gsize)
        lodexb = np.floor((lodb[:]+180.0) / gsize)
        
        iwpallb.extend(iwpb)
        iwpalls.extend(iwpsmoo)

        for x in range(nx):
            if np.any(lodexb == x):
                dexsubb = np.where(lodexb == x)
                subsetb = iwpb[dexsubb] # using smoothed iwp or not?
                subsetlab = ladexb[dexsubb]
                for y in range(ny):
                    if np.any(subsetlab == y):
                        lolasubsetb = np.where(subsetlab == y)
                        iwpgb[y,x] += subsetb[lolasubsetb].sum() # add up, get mean afterward
                        iwpgcb[y,x] += len(subsetb[lolasubsetb])
                        
            if np.any(lodex == x):
                dexsub = np.where(lodex == x)
                subset = iwpsmoo[dexsub] # using smoothed iwp or not?
                subsetla = ladex[dexsub]
                for y in range(ny):
                    if np.any(subsetla == y):
                        lolasubset = np.where(subsetla == y)
                        iwpg[y,x] += subset[lolasubset].sum() # add up, get mean afterward
                        iwpgc[y,x] += len(subset[lolasubset])
                        

DARDAR-CLOUD_v2.1.1_2008306010631_13364.h5 0
DARDAR-CLOUD_v2.1.1_2008307001055_13378.h5 1
DARDAR-CLOUD_v2.1.1_2008308005412_13393.h5 2
DARDAR-CLOUD_v2.1.1_2008309013730_13408.h5 3
DARDAR-CLOUD_v2.1.1_2008310004154_13422.h5 4
DARDAR-CLOUD_v2.1.1_2008311012511_13437.h5 5
DARDAR-CLOUD_v2.1.1_2008312002935_13451.h5 6
DARDAR-CLOUD_v2.1.1_2008313011252_13466.h5 7
DARDAR-CLOUD_v2.1.1_2008314001717_13480.h5 8
DARDAR-CLOUD_v2.1.1_2008315010034_13495.h5 9
DARDAR-CLOUD_v2.1.1_2008316000458_13509.h5 10
DARDAR-CLOUD_v2.1.1_2008317004815_13524.h5 11
DARDAR-CLOUD_v2.1.1_2008318013132_13539.h5 12
DARDAR-CLOUD_v2.1.1_2008319003556_13553.h5 13
DARDAR-CLOUD_v2.1.1_2008320011914_13568.h5 14
DARDAR-CLOUD_v2.1.1_2008321002338_13582.h5 15
DARDAR-CLOUD_v2.1.1_2008322010655_13597.h5 16
DARDAR-CLOUD_v2.1.1_2008323001119_13611.h5 17
DARDAR-CLOUD_v2.1.1_2008324005436_13626.h5 18
DARDAR-CLOUD_v2.1.1_2008325013753_13641.h5 19
DARDAR-CLOUD_v2.1.1_2008326004217_13655.h5 20
DARDAR-CLOUD_v2.1.1_2008327012534_13670.h5 2

In [ ]:
# turn into means 
realz = np.where(iwpgc >= 1)
iwp_mean = np.zeros([ny,nx])
iwp_mean[realz] = iwpg[realz] / iwpgc[realz] # calculate means
realzb = np.where(iwpgcb >= 1)
iwpb_mean = np.zeros([ny,nx])
iwpb_mean[realzb] = iwpgb[realzb] / iwpgcb[realzb] # calculate means

In [ ]:
#print(info(ladb),info(lodb))
#print(info(lat),info(lon))
print(info(iwp_mean[realz]))
print(info(iwpb_mean[realzb]))
#print(info(iwpg),info(iwpgc))
#print(info(iwpgb),info(iwpgcb))
print(info(iwpb),info(iwpsmoo),info(ariwp))
np.shape(iwpgc),np.shape(iwpg),np.shape(iwpsmoo)


In [ ]:
print(info(iwp_mean))
fig = plt.figure(figsize=[16,11])
#iwp_mean[np.where(iwp_mean > 4)] = 4.0 # say
#from dmap import grdmap
mapz = grdmap(np.flipud(iwp_mean),-180,startlat,.001)
#fig.savefig('dardar.10days.1deg.jpg',dpi=300)
#plt.imshow(iwp_mean)
#plt.show()
fig2 = plt.figure(figsize=[16,11])
mapz = grdmap(np.flipud(iwpb_mean),-180,startlat,.001)

In [ ]:
# create histogram of IWP

#b = np.histogram(iwp,bins=[0.001,.005,.01,.05,.1,.5,1,2,4,8])
#binz = [0.0,.0001,.00025,.0005,0.001,.0025,.005,.01,.025,.05,.1,.25,.5,1,2,4,8]
binz = [2**x for x in range(-14,3)]
latah = np.array(lata)
dnaa = np.array(dna)
iwpa = np.array(iwpall)
hist, bin_edges = np.histogram(iwpa,bins=binz)#,density=True)
iwpab = np.array(iwpallb)
iwpas = np.array(iwpalls)
histb, bin_edges = np.histogram(iwpab,bins=binz)#,density=True)
hists, bin_edges = np.histogram(iwpas,bins=binz)#,density=True)
mask1 = abs(latah)>30.0
mask2 = abs(latah)<30.0
mask3 = dnaa==0
mask4 = dnaa==1
iw1 = iwpa[mask1] #high lats
iw2 = iwpa[mask2] #tropics
iw3 = iwpa[mask3]
iw4 = iwpa[mask4]
hist1, bin_edges = np.histogram(iw1,bins=binz)#,density=True)
hist2, bin_edges = np.histogram(iw2,bins=binz)#,density=True)
hist3, bin_edges = np.histogram(iw3,bins=binz)#,density=True)
hist4, bin_edges = np.histogram(iw4,bins=binz)#,density=True)
fig = plt.figure()
ax = fig.add_subplot(111)

np.size(iwpa)
numbins = len(hist)
plt.plot(binz[1:],hist/sum(hist),'k-o',label='All (Native)')
#plt.plot(binz[1:],hist1,'r-o',label='Lat >30')
#plt.plot(binz[1:],hist2,'g-o',label='Lat <30')
#plt.plot(binz[1:],hist3,'b-o',label='Daytime')
#plt.plot(binz[1:],hist4,'y-o',label='Nighttime')
plt.plot(binz[1:],histb/sum(histb),'r--o',label='Chunked')
plt.plot(binz[1:],hists/sum(hists),'r-o',label='Smoothed')
#plt.yscale('log')
plt.xscale('log')
plt.xlabel(r'$IWP [kg/m^2]$')
plt.ylabel('Counts')
#plt.ylim(.1,1.1)
plt.legend()
plt.title('IWP histo DARDAR, '+str(len(iwpa))+'pts')#+fii)
#fig.savefig('dardar'+str(len(iwpa))+'.png',dpi=300)
plt.show()
